In [1]:
import sqlite3
import csv
import random
import pandas as pd
import re
from tqdm import tqdm

In [94]:
# 트위치 용

In [114]:
con = sqlite3.connect("FinalTwitchDb.db")
cur = con.cursor()

In [14]:
def make_csv2(f_name,n_url=3000):
    pattern1 = re.compile(r"[.,\"']") 
    cur.execute("SELECT content, url_id, id FROM chat WHERE id NOT IN (SELECT sampled_id FROM sampling) LIMIT ?",[n_url])
    result = cur.fetchall()
    f = open('{}.csv'.format(f_name),'w',encoding='utf-16',newline='')
    wr = csv.writer(f)
    wr.writerow(['comment'+'\t'+'url_id'])
    for _ in tqdm(result):
        cur.execute("INSERT INTO sampling(sampled_id) VALUES(?)",[_[2]])
        con.commit()
        rep = pattern1.sub("",_[0])
        wr.writerow([rep])
    f.close()

In [116]:
# sapling table 비워주기

cur.execute("DELETE FROM sampling;")

con.commit()

In [ ]:
# 엑셀은 최대 104만개 행만 가능. 따로 분리해주는 작업 필요함
# 엑셀 다른 이름으로 저장하기 하면 BOM 생성되서 코드 실행 에러남. 그냥 해당 파일 여러 개 복사해서 각 파일로 공백 메꾸기 작업하고
# 그 다음 메모장으로 다시 합쳐서 200만개로 쓰기

In [117]:
make_csv2('트위치200만', 2000000)

100%|█████████████████████████████████████████████████████████████████████| 2000000/2000000 [2:27:43<00:00, 225.65it/s]


In [118]:
con.close()

In [2]:
import re
from konlpy.tag import Okt
okt = Okt()

def prep1(textt = '존ㄴ ㅂ신 병ㅅ 새ㄲ ㅈ나 ㅅ발 ㅁㅊ놈아'):
    pattern2 = re.compile(r'[^ a-zA-Zㄱ-ㅊㅌ-ㅠㅢ가-힣]')
    textt = pattern2.sub('',textt)
    temp_okt = okt.pos(textt) # , norm=True, stem=True
    
    cut_word_h = []
    cut_word_e = []
    temp = []
    
    for a in range(len(temp_okt)):
        if temp_okt[a][1] == 'Alpha':
            cut_word_e.append(temp_okt[a])
        else:
            cut_word_h.append(temp_okt[a])
    
    cut_word_h.extend(cut_word_e)
    temp.extend(cut_word_h)
    
    temp2 = []
    string1 = ''
    k = -1

    # 병신, 시발, 씨발, 존나, 씨발롬들아, 미친놈아, 새끼, 좆밥
    bad_word = ['ㅂ', 'ㅅ', 'ㅆ', 'ㅈ', 'ㅁㅊ', 'ㄲ', 'ㄴ']
    bad_set_word = ['신', '발', '빨', '나', '놈', '끼', '새', '존', '밥', '문대']
    important_class = ['KoreanParticle', 'Noun', 'mixed', 'Verb', 'Adjective', 'Alpha']
    not_important_word = ['가']

    for i in range(len(temp)):
        if temp[i][0] in bad_word:
            if i < len(temp) - 1 and len(temp) > 1 and temp[i+1][0] in bad_set_word:
                if temp[i+1][1] == 'Noun' and temp[i+1][0] not in not_important_word:
                    for j in range(i):
                        if j <= k or temp[j][1] not in important_class or (len(temp[j][0]) > 6 and temp[j][1] == 'Verb') or temp[j][0] in not_important_word:
                            continue
                        temp2.append(temp[j])
                    string1 = temp[i][0] + temp[i+1][0]
                    temp2.append((string1, 'mixed'))
                    k = i+1
            else:
                if i != 0 and len(temp[i-1][0]) == 1 and  temp[i-1][1] == 'Noun' and temp[i-1][0] not in not_important_word:
                    for j in range(i):
                        if j <= k or temp[j][1] not in important_class or (len(temp[j][0]) > 6 and temp[j][1] == 'Verb') or temp[j][0] in not_important_word:
                            continue
                        temp2.append(temp[j])
                    string1 = temp[i-1][0] + temp[i][0]
                    temp2.pop()
                    temp2.append((string1, 'mixed'))
                    k = i

        if i < len(temp) - 1 and len(temp) > 1:
            if temp[i][1] == 'Noun' and temp[i+1][1] == 'Suffix' and temp[i+1][0] not in not_important_word:
                for j in range(i):
                    if j <= k or temp[j][1] not in important_class or (len(temp[j][0]) > 6 and temp[j][1] == 'Verb') or temp[j][0] in not_important_word:
                        continue
                    temp2.append(temp[j])
                string1 = temp[i][0] + temp[i+1][0]
                temp2.append((string1, 'Noun + Suffix'))
                k = i+1
                
        # 형태소분석기가 인식못하는 예외단어(2)
        if i < len(temp) - 1 and len(temp) > 1:
            if (temp[i][0] == '운' and temp[i+1][0] == '지') or (temp[i][0] == '노' and temp[i+1][0] == '무현')\
            or (temp[i][0] == '찐' and temp[i+1][0] == '따') or (temp[i][0] == '섹' and temp[i+1][0] == '히')\
            or (temp[i][0] == '새' and temp[i+1][0] == '키') or (temp[i][0] == '새' and temp[i+1][0] == '퀴')\
            or (temp[i][0] == '도' and temp[i+1][0] == '네') or (temp[i][0] == '장애' and temp[i+1][0] == '인색히')\
            or (temp[i][0] == '자' and temp[i+1][0] == '지') or (temp[i][0] == '고' and temp[i+1][0] == '소')\
            or (temp[i][0] == '저능' and temp[i+1][0] == '아') or (temp[i][0] == '색' and temp[i+1][0] == '히')\
            or (temp[i][0] == '쓰래' and temp[i+1][0] == '기') or (temp[i][0] == '애' and temp[i+1][0] == '미')\
            or (temp[i][0] == 'ㅅ' and temp[i+1][0] == 'ㅂ') or (temp[i][0] == 'ㅂ' and temp[i+1][0] == 'ㅅ')\
            or (temp[i][0] == 'ㅈ' and temp[i+1][0] == 'ㄹ') or (temp[i][0] == 'ㅈ' and temp[i+1][0] == 'ㄴ')\
            or (temp[i][0] == 'ㅁ' and temp[i+1][0] == 'ㅊ') or (temp[i][0] == 'ㅅ' and temp[i+1][0] == 'ㄲ')\
            or (temp[i][0] == 'ㅈ' and temp[i+1][0] == 'ㅂ') or (temp[i][0] == '섯' and temp[i+1][0] == '네')\
            or (temp[i][0] == '섰' and temp[i+1][0] == '네') or (temp[i][0] == '섯' and temp[i+1][0] == '다')\
            or (temp[i][0] == '섰' and temp[i+1][0] == '다') or (temp[i][0] == '시' and temp[i+1][0] == '발')\
            or (temp[i][0] == '새' and temp[i+1][0] == '끼') or (temp[i][0] == '개간' and temp[i+1][0] == '년')\
            or (temp[i][0] == '시부' and temp[i+1][0] == '렬') or (temp[i][0] == '쳐' and temp[i+1][0] == '받는')\
            or (temp[i][0] == '닝' and temp[i+1][0] == '기리') or (temp[i][0] == '개' and temp[i+1][0] == '공감')\
            or (temp[i][0] == '개' and temp[i+1][0] == '새기') or (temp[i][0] == '빠' and temp[i+1][0] == '가')\
            or (temp[i][0] == '개' and temp[i+1][0] == '소리') or (temp[i][0] == '개' and temp[i+1][0] == '쉽다')\
            or (temp[i][0] == '개' and temp[i+1][0] == '쉽네') or (temp[i][0] == '개' and temp[i+1][0] == '싫다')\
            or (temp[i][0] == '계' and temp[i+1][0] == '새끼') or (temp[i][0] == '김치' and temp[i+1][0] == '녀')\
            or (temp[i][0] == '에' and temp[i+1][0] == '라이') or (temp[i][0] == '관' and temp[i+1][0] == '종')\
            or (temp[i][0] == '괘' and temp[i+1][0] == '새끼') or (temp[i][0] == '그' and temp[i+1][0] == '켬')\
            or (temp[i][0] == '꼬' and temp[i+1][0] == '라지') or (temp[i][0] == '노' and temp[i+1][0] == '알라')\
            or (temp[i][0] == '노친' and temp[i+1][0] == '네') or (temp[i][0] == '눈' and temp[i+1][0] == '깔')\
            or (temp[i][0] == '니' and temp[i+1][0] == '년') or (temp[i][0] == '이' and temp[i+1][0] == '년')\
            or (temp[i][0] == '머' and temp[i+1][0] == '갈') or (temp[i][0] == '덜' and temp[i+1][0] == '떨어진')\
            or (temp[i][0] == '시' and temp[i+1][0] == '바알') or (temp[i][0] == '또' and temp[i+1][0] == '라인')\
            or (temp[i][0] == '시' and temp[i+1][0] == '펄') or (temp[i][0] == '디' and temp[i+1][0] == '쥐고싶냐')\
            or (temp[i][0] == '병' and temp[i+1][0] == '신') or (temp[i][0] == '존' and temp[i+1][0] == '나')\
            or (temp[i][0] == '섹' and temp[i+1][0] == '스') or (temp[i][0] == '니' and temp[i+1][0] == '미')\
            or (temp[i][0] == '미' and temp[i+1][0] == '친') or (temp[i][0] == '느' and temp[i+1][0] == '그')\
            or (temp[i][0] == '어' and temp[i+1][0] == '미') or (temp[i][0] == '느' and temp[i+1][0] == '금')\
            or (temp[i][0] == '메' and temp[i+1][0] == '갈') or (temp[i][0] == '일' and temp[i+1][0] == '베')\
            or (temp[i][0] == '병' and temp[i+1][0] == '자') or (temp[i][0] == '장' and temp[i+1][0] == '애')\
            or (temp[i][0] == '씨' and temp[i+1][0] == '발') or (temp[i][0] == '븽' and temp[i+1][0] == '신')\
            or (temp[i][0] == '빨' and temp[i+1][0] == '아') or (temp[i][0] == '새' and temp[i+1][0] == '낀데')\
            or (temp[i][0] == '개' and temp[i+1][0] == '같이') or (temp[i][0] == '슴' and temp[i+1][0] == '가')\
            or (temp[i][0] == '정신병' and temp[i+1][0] == '자') or (temp[i][0] == '친구' and temp[i+1][0] == '년')\
            or (temp[i][0] == '오' and temp[i+1][0] == '지구') or (temp[i][0] == '오' and temp[i+1][0] == '쪘')\
            or (temp[i][0] == '머리' and temp[i+1][0] == '텅') or (temp[i][0] == '쪼' and temp[i+1][0] == '다')\
            or (temp[i][0] == '디' and temp[i+1][0] == '질') or (temp[i][0] == '뒤' and temp[i+1][0] == '질')\
            or (temp[i][0] == '디' and temp[i+1][0] == '졌') or (temp[i][0] == '딴' and temp[i+1][0] == '년')\
            or (temp[i][0] == '또' and temp[i+1][0] == '라이') or (temp[i][0] == '똘' and temp[i+1][0] == '아이')\
            or (temp[i][0] == '오' and temp[i+1][0] == '쪘') or (temp[i][0] == '명존' and temp[i+1][0] == '쎄')\
            or (temp[i][0] == '미시' and temp[i+1][0] == '친발') or (temp[i][0] == '주' and temp[i+1][0] == '겨')\
            or (temp[i][0] == '윾' and temp[i+1][0] == '두') or (temp[i][0] == '기' and temp[i+1][0] == '무띠')\
            or (temp[i][0] == '죽여뿌' and temp[i+1][0] == '고') or (temp[i][0] == '미' and temp[i+1][0] == '틴')\
            or (temp[i][0] == '별' and temp[i+1][0] == '창') or (temp[i][0] == '색' and temp[i+1][0] == '희')\
            or (temp[i][0] == '수준' and temp[i+1][0] == '하고는') or (temp[i][0] == '쉬이' and temp[i+1][0] == '바')\
            or (temp[i][0] == '시미' and temp[i+1][0] == '발친') or (temp[i][0] == '시미' and temp[i+1][0] == '친발')\
            or (temp[i][0] == '시' and temp[i+1][0] == '부울') or (temp[i][0] == '시방' and temp[i+1][0] == '새')\
            or (temp[i][0] == '시불' and temp[i+1][0] == '탱') or (temp[i][0] == '시' and temp[i+1][0] == '빨')\
            or (temp[i][0] == '시' and temp[i+1][0] == '이발') or (temp[i][0] == '개시' and temp[i+1][0] == '키')\
            or (temp[i][0] == '시' and temp[i+1][0] == '팔') or (temp[i][0] == '십' and temp[i+1][0] == '창')\
            or (temp[i][0] == '쓰' and temp[i+1][0] == '렉') or (temp[i][0] == '씨' and temp[i+1][0] == '바라')\
            or (temp[i][0] == '씨방' and temp[i+1][0] == '새') or (temp[i][0] == '개' and temp[i+1][0] == '년')\
            or (temp[i][0] == '개' and temp[i+1][0] == '같네') or (temp[i][0] == '개' and temp[i+1][0] == '같아')\
            or (temp[i][0] == '개' and temp[i+1][0] == '같다') or (temp[i][0] == '처' and temp[i+1][0] == '먹어')\
            or (temp[i][0] == '처' and temp[i+1][0] == '먹다') or (temp[i][0] == '처' and temp[i+1][0] == '먹여')\
            or (temp[i][0] == '씨' and temp[i+1][0] == '이발') or (temp[i][0] == '씨' and temp[i+1][0] == '바알')\
            or (temp[i][0] == '졀' and temp[i+1][0] == '라') or (temp[i][0] == '졌' and temp[i+1][0] == '같은')\
            or (temp[i][0] == '조' and temp[i+1][0] == '낸') or (temp[i][0] == '족' and temp[i+1][0] == '까')\
            or (temp[i][0] == '존' and temp[i+1][0] == '쎄') or (temp[i][0] == '좁' and temp[i+1][0] == '밥')\
            or (temp[i][0] == '종' and temp[i+1][0] == '나') or (temp[i][0] == '싸' and temp[i+1][0] == '물어')\
            or (temp[i][0] == '닭' and temp[i+1][0] == '쳐') or (temp[i][0] == '닦' and temp[i+1][0] == '쳐')\
            or (temp[i][0] == '닭' and temp[i+1][0] == '근혜') or (temp[i][0] == '쥐박' and temp[i+1][0] == '이')\
            or (temp[i][0] == '죵' and temp[i+1][0] == '나') or (temp[i][0] == '줬' and temp[i+1][0] == '같은')\
            or (temp[i][0] == '쥰' and temp[i+1][0] == '나') or (temp[i][0] == '쥰' and temp[i+1][0] == '니')\
            or (temp[i][0] == '지' and temp[i+1][0] == '롤') or (temp[i][0] == '와' and temp[i+1][0] == '꾸')\
            or (temp[i][0] == '짱' and temp[i+1][0] == '께') or (temp[i][0] == '존' and temp[i+1][0] == '맛')\
            or (temp[i][0] == '쳐' and temp[i+1][0] == '먹어') or (temp[i][0] == '쳐' and temp[i+1][0] == '먹다')\
            or (temp[i][0] == '쳐' and temp[i+1][0] == '발라') or (temp[i][0] == '쳐' and temp[i+1][0] == '바르다')\
            or (temp[i][0] == '개' and temp[i+1][0] == '사기')\
            :
                for j in range(i):
                    if j <= k or temp[j][1] not in important_class or (len(temp[j][0]) > 6 and temp[j][1] == 'Verb') or temp[j][0] in not_important_word:
                        continue
                    temp2.append(temp[j])
                string1 = temp[i][0] + temp[i+1][0]
                temp2.append((string1, 'Except'))
                k = i+1
                
        # 형태소분석기가 인식못하는 예외단어(3)
        if i < len(temp) - 2 and len(temp) > 2:
            if (temp[i][0] == '앙' and temp[i+1][0] == '기' and temp[i+2][0] == '무')\
            or (temp[i][0] == '앙' and temp[i+1][0] == '기' and temp[i+2][0] == '무띠')\
            or (temp[i][0] == '보' and temp[i+1][0] == '딩고' and temp[i+2][0] == '지')\
            or (temp[i][0] == '앙' and temp[i+1][0] == '기모' and temp[i+2][0] == '띄')\
            or (temp[i][0] == '시' and temp[i+1][0] == '새' and temp[i+2][0] == '발끼')\
            or (temp[i][0] == '씨' and temp[i+1][0] == '새' and temp[i+2][0] == '발끼')\
            or (temp[i][0] == '시' and temp[i+1][0] == '친' and temp[i+2][0] == '발미')\
            or (temp[i][0] == '죠' and temp[i+1][0] == '온' and temp[i+2][0] == '나')\
            :
                for j in range(i):
                    if j <= k or temp[j][1] not in important_class or (len(temp[j][0]) > 6 and temp[j][1] == 'Verb') or temp[j][0] in not_important_word:
                        continue
                    temp2.append(temp[j])
                string1 = temp[i][0] + temp[i+1][0] + temp[i+2][0]
                temp2.append((string1, 'Except'))
                k = i+2
        
        # 형태소분석기가 인식못하는 예외단어(4)
        if i < len(temp) - 3 and len(temp) > 3:
            if (temp[i][0] == '앙' and temp[i+1][0] == '기' and temp[i+2][0] == '모' and temp[i+3][0] == '띠')\
            or (temp[i][0] == '앙' and temp[i+1][0] == '기' and temp[i+2][0] == '모' and temp[i+3][0] == '딱')\
            :
                for j in range(i):
                    if j <= k or temp[j][1] not in important_class or (len(temp[j][0]) > 6 and temp[j][1] == 'Verb') or temp[j][0] in not_important_word:
                        continue
                    temp2.append(temp[j])
                string1 = temp[i][0] + temp[i+1][0] + temp[i+2][0] + temp[i+3][0]
                temp2.append((string1, 'Except'))
                k = i+3
        
        # 예외단어(엿)
        if temp[i][0] == '엿'and temp[i][1] == 'Modifier':
            for j in range(i):
                if j <= k or temp[j][1] not in important_class or (len(temp[j][0]) > 6 and temp[j][1] == 'Verb') or temp[j][0] in not_important_word:
                    continue
                temp2.append(temp[j])
            string1 = temp[i][0]
            temp2.append((string1, 'Except'))
            k = i
            
    else:
        for j in range(len(temp)):
            if j <= k or temp[j][1] not in important_class or (len(temp[j][0]) > 6 and temp[j][1] == 'Verb') or temp[j][0] in not_important_word:
                continue
            temp2.append(temp[j])

    result = []
    
    for b in range(len(temp2)):
        result.append(temp2[b])
    
    return result

In [2]:
prep1('짱깨 짱꺠 양키 쪽바리 현피')

['짱깨', '짱꺠', '양키', '쪽바리', '현피']

In [3]:
def prep2(datasets):
    '''
    dataframe을 넣어주세요
    '''
    chat = list()
    for i,j in enumerate(datasets.comment.values):
        temp = []
        temp.append(prep1(j))
        if temp == [[]]:
            datasets.drop(i, inplace=True)
            continue
        else:
            chat.extend(temp)
    
    return chat

In [19]:
abc = prep2(['맨두', '고기맨두', '김치왕맨두'])

AttributeError: 'list' object has no attribute 'comment'

In [17]:
abc

[['맨'], ['맨'], ['김치왕', '맨']]

In [ ]:
chat = list()
for i,j in enumerate(datasets.comment.values):
    temp = []
    temp.append(prep1(j))
    if temp == [[]]:
        datasets.drop(i, inplace=True)
        continue
    else:
        chat.extend(temp)
chat[:5]

In [4]:
import numpy as np
import pandas as pd
import json
import re


def AUTO_LABEL(dataset,dictionary):
    '''
    인풋으로 '비속어사전'(dictionary)과 '필터링할 DataFrame'을 사용합니다.
     
    그러면 이 함수는 해당 dataFrame과 비속어사전을 매핑하여, 
    비속어사전에 있는 단어가 해당파일에 있는 경우에
    필터링된 파일(dataset)에 유해함 여부를 1로써 나타냅니다.
    
    이때 유해도를 나타내는 컬럼은 dataset에서 새로운 'filtered'에 나타나며
    filtered는 디폴트로 0 값을 갖고 있다가, 비속어 사전과 채팅의 데이터가 매핑되면
    유해도가 1로 변경됩니다.
      
    '''
# 1. 데이터 셋 불러오기
    datasets = pd.DataFrame()
    dump = pd.read_csv(dataset, sep = '\t', engine='python', encoding='utf-16')
    datasets = pd.concat([datasets, dump])
        
# 2. 비속어 사전 불러오기
    with open(dictionary,'r') as f:
        forbiddenword = json.load(f)
        list2=list(forbiddenword.values()) 
        dict=list2[0] 
        dict1="|".join(dict) 
        
        
# 3. filtered라는 컬럼을 데이터셋에 추가함. (디폴트0, 필터되어 유해하면 1로 변경)
    datasets['filtered']=0
    data = datasets.comment

# 4. 유해 여부 실질적으로 판단
    idx = [i for i,d in enumerate(data) if re.search(dict1, d)]   
    datasets.iloc[idx,-1] = 1
    return datasets
#     return datasets[datasets['filtered']==1]
#=>필터된거만 보고싶을때

In [282]:
# AUTO_LABEL('트위치채팅1.csv','word-fobidden.json')

In [5]:
from pandas import Series, DataFrame

In [6]:
datasets = pd.DataFrame()
dump = pd.read_csv('train.csv', engine='python', encoding='utf-16')
datasets = pd.concat([datasets, dump])

In [7]:
datasets

,Unnamed: 0,comment,url_id,label
0,0,ㅜㅜ,77,0
1,1,ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ,17,0
2,2,헐,52,0
3,3,제시,75,0
4,4,이거인 듯,18,0
5,5,개써글겜,52,0
6,6,ㅇㅈ,75,0
7,7,손 찧으면 진짜 아픈데,57,0
8,8,ㅇ우무 말고 몰왕,65,0
9,9,그건 아니던데,74,0


In [8]:
chat = list()
for i,j in tqdm(enumerate(datasets.comment.values)):
    temp = []
    temp.append(prep1(j))
    temp2 = ''
    for k in range(len(temp[0])):
        temp2 += temp[0][k] + ' '
    chat.append([temp2])

42000it [00:45, 929.77it/s] 


In [9]:
chat

[['ㅜㅜ '],
 [''],
 ['헐 '],
 ['제시 '],
 ['거인 듯 '],
 ['개써 글겜 '],
 ['ㅇㅈ '],
 ['손 찧으면 진짜 아픈데 '],
 ['ㅇ 우무 몰 왕 '],
 ['아니던데 '],
 ['노틸 '],
 [''],
 ['모드 충돌 '],
 ['방광 받음 안됨 '],
 [''],
 ['밥 '],
 ['궁금하시면 방송 직접 보세요 '],
 ['궁 순수 너프 '],
 ['ㄴㅇㅂㅈ '],
 ['람각 '],
 ['짜 고생 햇더 '],
 ['배 방구 '],
 ['엌 '],
 ['오우 '],
 ['어케햇누 '],
 [''],
 ['너구리 들어갔으면 너구리 죽었지 않음 '],
 ['뭐 '],
 ['루시안 것 하네 '],
 ['ㅁㅍ '],
 ['ㅈㄴ 뿡뿡이 '],
 ['유미 샌 박이었네 ㄷ '],
 ['즐기시게 냅 둬 '],
 ['걸 '],
 ['온플릭 살았어 '],
 ['왜 일까 요 '],
 [''],
 ['창 많은데 '],
 ['박스 깨 '],
 ['무빙 오지네 '],
 [''],
 ['개 웃겨 '],
 ['적장 목 '],
 ['세균 넘 모 맣ㄴ 은디 '],
 ['개 무섭지 '],
 ['ㅔ '],
 [''],
 ['데스 '],
 ['재밌어 욬 '],
 ['한국인 바로 빠따쳣다 ㄹㅇ '],
 ['잘은 좀 '],
 ['거 대본 이럴 리가 없음 '],
 ['님 스트리머 무한 존경 표 합니다 '],
 ['실드 채워 '],
 ['프로 때 말 안해 '],
 ['또 죽었네 '],
 ['동준 이형 진짜 화난거 같은데 말 하네 '],
 ['F L E X '],
 ['ㅠㅠㅠ '],
 ['페이커 혼자 각 봐서 호응 하기가 힘들어 '],
 ['소리 '],
 ['시 영태 '],
 ['큐베 만 들어준다 '],
 ['ㅏㅔ '],
 ['심층 연구 '],
 ['나와 '],
 ['ㅔㅔㅔ '],
 ['유 '],
 [''],
 [''],
 ['활 짝 '],
 ['귀신 인간성 찾다니 '],
 ['설거지 '],
 ['앗 히오스 '],
 ['하단 안 막 '],
 ['줄건 줘 '],
 

In [10]:
mandoo = DataFrame(chat, columns=['comment'])

In [11]:
mandoo

,comment
0,ㅜㅜ
1,
2,헐
3,제시
4,거인 듯
5,개써 글겜
6,ㅇㅈ
7,손 찧으면 진짜 아픈데
8,ㅇ 우무 몰 왕
9,아니던데


In [12]:
with open('word-fobidden.json','r') as f:
        forbiddenword = json.load(f)
        list2=list(forbiddenword.values()) 
        dict=list2[0] 
        dict1="|".join(dict) 

In [13]:
dict1

'jonna|ang|sibal|sex|fuck|suck|byungsin|wtf|shut|shit|ass|pussy|yello monkey|dick|boob|새끼|개새|쌉|개간년|년들|시부럴|시뷰럴|씨부럴|씨뷰럴|시부렬|시뷰렬|씨부렬|씨뷰렬|쳐받는|닝기리|스벌|호로|개공감|개년|빠가|개련|개망|개부랄|개새기|개새끼|새끼|개섹|섹|개소리|개쉽다|개쉽네|개싫다|개자식|계새끼|김치녀|맘충|엿같|존낙|젖|닥치세|닥쳐|에라이|퉤|슈벌|슈우벌|빻|오진다|오져|골빈|골빈|글러 먹|글러먹|오짐|관종|괘새끼|씹|그켬|극혐|꼬라지|꼴갑|꼴값|꼴깝|썅|썅놈|남혐|여혐|넌씨|년놈|노무|놈|노알라|노친네|눈깔|늬미|니년|니미럴|애미|자지|고츄|보지|운지|대가리|머갈|대갈|덜떨어진|도랏|도랐|시바알|도른|새끼|씨벌|돌았나|돌았네|뒈져|뒤져야|쓰레기|씹뻐럴|또라인|시펄|쉬펄|쉬버|미친|미쳤다|미쳣다|개꿀잼|암살|뇌절|뇌피셜|뇌텅|뒤져|씨뻘|뒤졌|디지고|ㅈ문대|개돼지|씹못|시바류|시바|슴가|졸라|오타쿠|씹덕|정신병자|새낀데|개같이|망해라|빡새끼|디져라|찌질한|ㅅㅂ|ㅆㅂ|ㅂㅅ|ㅄ|ㄱㅅㄲ|ㄳㄲ|ㅁㅊㄴ|ㅆㅂㄹㄴ|ㅅㅂㄹㄴ|쓰발|ㄷㅈㄹ|ㄷㅊ|ㅁㅊㅅㄲ|ㅈㄴ|ㅈㄲㅇㄴ|ㄴㅇㅁ|염병|ㅈ대로|ㄱㅈㄹ|ㅈㄹ|ㅈㄲ|ㅊㄴ|ㅅ발|시ㅂ|ㅆ발|씨ㅂ|ㅂ신|병ㅅ|ㅂㅈ|개ㅅㄲ|친구년|버러지|에반데|오지고|오지구|오지네|똥꼬|개웃기네|찐|오졌|오지는|오쪘|5졌|5지구|5지고|5지네|뇌텅|머리텅|쪼다|뒤지|뒤진다|뒤질|디졌|디질|딴년|도라이|또라이|똘아이|듣보|런년|ㅁㅊ|ㅁ친|미ㅊ|명존쎄|주겨|윾두|보딩고지|섹도시발|앙기무띠|앙기모딱|앙기모띠|앙기모띄|기모찌|미시친발|미쳤나|미쳤니|미췬|죽여버리고|죽여뿌고|미틴|및힌|별창|병맛|병신|병크|븅신|빠큐|뻐규|뻐큐|뻑유|뻑큐|빠따|뼈큐|쉨|쉑|뽄새|뽄세|ㅅㅋ|새키|색희|색히|샹년|쌍년|썅년|섀키|성괴|수준하고는|쉬이바|슈발|슈레기|슨상님|시미발친|시미친발|시발|시새발끼|씨새발끼|시부울|시방새|시벌탱|시볼탱|시빨|시이발|시친발미|개시키|사망|살인|시팔|거지같은|거지같네|ㅗㅗ|개련|꺼져|

In [293]:
# for i,d in tqdm(enumerate(data)):
#     if re.search(dict1, d):
#         print(d, i)







0it [00:00, ?it/s]

빠따 캐리 ㅇㅈ  391
세주 온다 ㅁㅊ  481
속보 노무현 사망 속보 노무현 사망 속보 노무현 사망 속보 노무현 사망 속보 노무현 사망 속보 노무현 사망 속보 노무현 사망 속보 노무현 사망  656
미친 새끼  674
올해 들어거 제일 개소리 같샤  677
형 오늘 밥 뭐처 드셨냐 고요 개새끼 몇번 물어봐요  685
섹  717
스타 듀 벨리 하지말고 제발 다른거하자 씨발  720
씨발 진짜 도네 하니까 척도 하네 창석 이형 스타 듀 벨리 끄자  723
중독 새끼  727
섹  752
헐 ㅁㅊ 진 엔딩 보기하 ㄷㄷ  823
사적 놈  902
북괴 놈 인성 봐  956
용인 에버랜드 머리 속 들어 찬 놈들 물어 뜯었던거네  994
ㅗㅗㅗㅗ  1044
훈남오빠 미친  1103
빠가닉  1238
입딜 미친다  1254
코 새끼들  1408
배 놈 또  1447
존나 드러  1503
ㅁㅊ 딜봐  1667








1668it [00:00, 16646.44it/s]

ㅈㄹㄴ  1903
어그 끈애 니 쓰레기  1911
진짜 도키 그날 수금 쌉 가능 같은데  1938








2000it [00:00, 16458.09it/s]

In [14]:
datasets['filtered']=0
data = mandoo.comment

idx = [i for i,d in tqdm(enumerate(data)) if re.search(dict1, d)]   
datasets.iloc[idx,-1] = 1

42000it [00:00, 132012.59it/s]


In [15]:
len(idx)

1277

In [16]:
datasets

,Unnamed: 0,comment,url_id,label,filtered
0,0,ㅜㅜ,77,0,0
1,1,ㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋㅋ,17,0,0
2,2,헐,52,0,0
3,3,제시,75,0,0
4,4,이거인 듯,18,0,0
5,5,개써글겜,52,0,0
6,6,ㅇㅈ,75,0,0
7,7,손 찧으면 진짜 아픈데,57,0,0
8,8,ㅇ우무 말고 몰왕,65,0,0
9,9,그건 아니던데,74,0,0


In [ ]:
datasets.to_csv('./test4')

In [17]:
import csv
import pandas as pd

In [20]:
def make_csv(f_name):
    f = open('{}.csv'.format(f_name),'w',encoding='utf-16',newline='')
    wr = csv.writer(f)
    wr.writerow(['comment'+','+'label'])
    for i in tqdm(range(len(datasets.comment))):
        wr.writerow([datasets.comment[i]+','+str(datasets.filtered[i])])
    f.close()

In [21]:
make_csv('test4만라벨')

100%|█████████████████████████████████████████████████████████████████████████| 42000/42000 [00:01<00:00, 22164.34it/s]
